### Importing Packages

In [1]:
import sqlite3
import pandas as pd
import numpy as np

### Global Variables

In [2]:
data_path = 'data/'
database = f'{data_path}dialects_database.db'

## Data Fetching

In [3]:
con = sqlite3.connect(database)

In [4]:
cur = con.cursor()

In [5]:
print(cur.execute("SELECT name FROM sqlite_master").fetchall())
cur.close()

[('id_text',), ('id_dialect',)]


In [6]:
def get_column_names(cursor : sqlite3.Cursor, table_name : str) -> list[str]:
    """
    Retrieve the column names for a specific table in the database.

    Parameters
    ----------
    cursor : sqlite3.Cursor
        Cursor object to execute SQL queries.
    table_name : str
        Name of the table to retrieve column names from.

    Returns
    -------
    list of str
        List of column names.
    """
    conn = sqlite3.connect(database)
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM {table_name}")

    #### Get column names from the cursor's description attribute
    column_names = [description[0] for description in cursor.description]

    conn.close()

    return column_names

print('Testing get_column_names function:')
table_name = 'sqlite_master'

cur = con.cursor()
print(f'The column names of $ {table_name} $ are : {get_column_names(cur, table_name)}')
cur.close()

Testing get_column_names function:
The column names of $ sqlite_master $ are : ['type', 'name', 'tbl_name', 'rootpage', 'sql']


In [7]:
def get_all_tables_in_db(cursor : sqlite3.Cursor) -> list[str]:
    """
    Retrieve the names of all tables in the database.

    Parameters
    ----------
    cursor : sqlite3.Cursor
        Cursor object to execute SQL queries.

    Returns
    -------
    list of str
        List of table names.
    """
    res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

    tables = [tab[0] for tab in res]

    return tables

print('Testing get_all_tables_in_db function:')
cur = con.cursor()
print(f'All the tables are : {get_all_tables_in_db(cur)}')
cur.close()

Testing get_all_tables_in_db function:
All the tables are : ['id_text', 'id_dialect']


In [8]:
def convert_database_tables_to_dataframe(database : str):
    """
    Extract data from the database and put it in a DataFrame

    Parameters
    ----------
    database : str
        Path to the SQLite database file.

    Returns
    -------
    list of pd.DataFrames
        List of the database tables in the dataframe format
    """
    conn = sqlite3.connect(database)
    cursor = conn.cursor()

    #### Get all tables in the database.
    table_names = get_all_tables_in_db(cursor)

    dfs = []

    #### Iterate over the tables and create a DataFrame for each table.
    for table_name in table_names:
        #### Get column names
        column_names = get_column_names(cursor, table_name)

        #### Get the data from the table
        text_cur = cursor.execute(f'SELECT * from {table_name}')
        text = text_cur.fetchall()

        #### Create a DataFrame with the column names
        text_df = pd.DataFrame(text, columns=column_names)

        print(text_df.head(3))

        dfs.append(text_df)

    cursor.close()

    return dfs

In [9]:
tables_df = convert_database_tables_to_dataframe(database)

                    id                                               text
0  1009754958479151232  @toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...
1  1009794751548313600  @AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...
2  1019989115490787200  @smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...
                    id dialect
0  1009754958479151232      LY
1  1009794751548313600      LY
2  1019989115490787200      LY


In [10]:
text, dialect = tables_df[0], tables_df[1]
assert text.shape[0] == dialect.shape[0], 'Make sure the tables has the same number of rows'

In [11]:
df_ = text.merge(dialect, on='id')
df_.drop(columns=['id'], inplace=True)
print(df_.shape)
df_.sample(3)

(147725, 2)


,text,dialect
24791,@farah25193 مشاء الله طالعه تجنني 😂😂😂,LY
63787,@hassan_almasriy آه \nالخميس المظلووووم😭\nحتى ...,EG
23750,- دزيلي صورتك بلا مكياج . \n- باهي ما تقول انك...,LY


## Saving and Loading the data

In [12]:
df_.to_excel(f'{data_path}arabic_dialects.xlsx', header=True, index=False)

In [14]:
pd.read_excel(f'{data_path}arabic_dialects.xlsx')

,text,dialect
0,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...,LY
1,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...,LY
2,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...,LY
3,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا...,LY
4,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ا...,LY
...,...,...
147720,الناس دي بتنفخ في قربة مقدودة بالدارجي كده ال...,SD
147721,@Wail_Hamza @amiromer انت عايش وين بره السودان...,SD
147722,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وا...,SD
147723,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وي...,SD


When we try to load the data after saving it using CSV format it results in the following error:


```
ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.
```
We can solve this error using open in `rb` mode and `io` the built-in library.

We decided to save the data in XlSX format.



_________

### Testing

In [15]:
df_.to_csv('arabic_dialects.csv', header=True, index=False, encoding='utf-8')

In [16]:
pd.read_csv('arabic_dialects.csv')

,text,dialect
0,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...,LY
1,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...,LY
2,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...,LY
3,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا...,LY
4,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ا...,LY
...,...,...
147720,الناس دي بتنفخ في قربة مقدودة بالدارجي كده ال...,SD
147721,@Wail_Hamza @amiromer انت عايش وين بره السودان...,SD
147722,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وا...,SD
147723,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وي...,SD


In [17]:
import io

# Read the file content as a string
with open('arabic_dialects.csv', 'r', encoding='utf-8', errors='ignore') as f:
    content = f.read()
    print(content[:500])
# Use StringIO to read the content
data = io.StringIO(content)
try:
    df = pd.read_csv(data)
    print(df.shape)
    print(df.columns)
    print(df.head())
except pd.errors.ParserError as e:
    print(f"Error parsing CSV: {e}")


text,dialect
@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
"@AlmFaisal 😂😂 الليبيين متقلبين!!!
بس بالنسبة ليا انا ميليشياوي زمان وتوة",LY
"@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
and they live happily ever after
ذي اند",LY
@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الان
(147725, 2)
Index(['text', 'dialect'], dtype='object')
                                                text dialect
0  @toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...      LY
1  @AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...      LY
2  @smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...      LY
3  @AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا...      LY
4  @lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ا...      LY


In [18]:
!pip install chardet


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import chardet

with open('arabic_dialects.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))

print(result)

with open('arabic_dialects.csv', 'r', encoding=result['encoding'], errors='ignore') as f:
    content = f.read()

data = io.StringIO(content)
try:
    df = pd.read_csv(data)
    print(df.shape)
    print(df.head())
except pd.errors.ParserError as e:
    print(f"Error parsing CSV: {e}")


{'encoding': 'TIS-620', 'confidence': 0.29502373875743676, 'language': 'Thai'}
(147725, 2)
                                                text dialect
0  @toha_Altomy @gy_yah ูููููู ุงุฏุจ ููู...      LY
1  @AlmFaisal ๐๐ ุงูููุจููู ูุชููุ...      LY
2  @smsm071990 @ALMOGRBE ูู 20 ุชุงููู ุดุงุ...      LY
3  @AboryPro @lyranoo85 ุฑุงููุง ุนูููุชู ู...      LY
4  @lyranoo85 ุดููู ูุชุนูุฏุฉ ุนูุดุงู ุง...      LY


-----------